In [10]:
import os
import zipfile
from pathlib import Path
import urllib.request as request
from dataclasses import dataclass
from src.cnnClassifier.logger import logging
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import *

In [9]:
pwd

'c:\\Users\\Cynthia\\Documents\\Learning Data Science\\projects\\chicken-disease\\chicken-disease'

In [3]:
os.chdir('../')

In [11]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [15]:
class configurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self):
        config=self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config=DataIngestionConfig(
                root_dir=config.root_dir,
                source_url=config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [13]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

    def download_file(self, filepath: Path):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(url=self.config.source_url, filename=self.config.local_data_file)

            logging.info(f"{filename} download! with following info \n{headers}")
        else:
            logging.info(f"file already exist of size {get_size(Path(self.config.local_data_file))}")


    def extract_zipfile(self):

        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [14]:
try:
    config=configurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingetion=DataIngestion(config=data_ingestion_config)
    data_ingetion.extract_zipfile()
except Exception as e:
    raise e